## 모델 훑어 보기


#### 모델의 구조
입력: [CLS]질문[SEP]지문[SEP]
출력: [정답의 시작일 확률, 끝일 확일]


## 모델 학습하기

#### 설정하기

In [1]:
# TPU 사용시 
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [2]:
!pip install ratsnlp

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 655 kB/s 
     |████████████████████████████████| 2.8 MB 12.6 MB/s 
     |████████████████████████████████| 806 kB 68.0 MB/s 
     |████████████████████████████████| 57 kB 5.7 MB/s 
     |████████████████████████████████| 8.0 MB 32.9 MB/s 
     |████████████████████████████████| 398 kB 75.1 MB/s 
     |████████████████████████████████| 140 kB 94.3 MB/s 
     |████████████████████████████████| 636 kB 55.8 MB/s 
     |████████████████████████████████| 829 kB 61.8 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x393d0000 @  0x7f895a75f615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 

In [3]:
#모델 환경 설정
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
#QATrainArguments 4장과 동일 
args = QATrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="korquad-v1",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
    #max_query_length 질문의 최대 길이
    doc_stride=64,
    #doc_stride 지문에서 몇 개 토큰을 슬라이딩하면서 데이터를 늘릴지 결정
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [4]:
#랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [5]:
#로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=4, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, fp16=False, tpu_cores=0, tqdm_enabled=True)


#### 말뭉치 내려받기

In [6]:
#말뭉치 내려받기
nlpbook.download_downstream_dataset(args)

Downloading: 38.5MB [00:00, 112MB/s]
Downloading: 3.88MB [00:00, 87.1MB/s]                  


#### 토크나이저 준비하기

In [7]:
#토크나이저 준비하기
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

#### 데이터 전처리

In [8]:
#학습데이터셋과 데이터로더 구축
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = KorQuADV1Corpus()
#KorQuADV1Corpus QADataset이 요구하면 QAExample을 제공
#QAExample 구성
  ## question_text
  ## context_text
  ##answer_text
  ##start_position_chatacter(정답의 시작 인덱스)
train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from train dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 57688/57688 [02:56<00:00, 327.55it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? [SEP] 18 ##3 ##9년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽고 그 내용 ##에 마음이 끌려 이를 소재 ##로 해서 하나의 교 ##향 ##곡 ##을 쓰 ##려는 뜻을 갖 ##는다 . 이 시기 바 ##그 ##너 ##는 18 ##3 ##8년 ##에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 걲 ##은 상황이 ##라 좌 ##절 ##과 실망 ##에 가득 ##했 ##으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만나는 파 ##우스 ##트 ##의 심 ##경 ##에 공감 ##했다고 한다 . 또한 파리 ##에서 아 ##브 ##네 ##크 ##의 지휘 ##로 파리 음악 ##원 관 ##현 ##악 ##단이 연 ##주 ##하는 베 ##토 [SEP]
INFO:ratsnlp:answer: 교 ##향 ##곡
INFO:ratsnlp:features: QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 10966

In [9]:
#평가용 데이터 로더 구축
val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from val dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 5533/5533 [00:17<00:00, 321.17it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 198 ##9년 6월 30 ##일 평양 ##축 ##전에 대표로 파견 된 인물 ##은 ? [SEP] 198 ##9년 2월 15일 여의도 농민 폭력 시위를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처벌 ##에 ##관 ##한 ##법 ##률 ##위반 ) 으로 지명 ##수 ##배 ##되었다 . 198 ##9년 3월 12 ##일 서울 ##지 ##방 ##검찰 ##청 공안 ##부는 임종석 ##의 사전 ##구속 ##영장 ##을 발부 ##받았 ##다 . 같은 해 6월 30 ##일 평양 ##축 ##전에 임 ##수 ##경을 대표로 파견 ##하여 국가보안법 ##위반 혐의가 추가 ##되었다 . 경찰은 12월 18 ##일 ~ 20 ##일 사이 서울 경 ##희 ##대학 ##교 ##에서 임종석 ##이 성 ##명 발표 ##를 추진 ##하고 있다는 첩 ##보를 입 ##수 ##했고 , 12월 18 ##일 오전 7시 40 ##분 경 가스 ##총 ##과 [SEP]
INFO:ratsnlp:answer: 임 ##수 ##경을
INFO:ratsnlp:features: QAFeatures(input_ids=[2, 25704, 29697, 23343, 8476, 4046, 14601, 4728, 8409, 24345, 20158, 902, 10393, 4057, 32, 3, 25704, 29697, 18163, 26757, 20604, 29804, 10901, 25099, 13225, 4047, 16704, 11, 10901, 12175, 4086, 11417, 4113, 4337

#### 모델 불러오기

In [10]:
# 모델 초기화
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

In [11]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model c

#### 모델 학습시키기

In [12]:
#테스크 정의
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)
#트레이너 정의
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [13]:
#학습
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                     | Params
---------------------------------------------------
0 | model | BertForQuestionAnswering | 108 M 
---------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.318   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

##학습 마친 모델을 실전 투입하기

In [ ]:
# !pip install ratsnlp

# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

In [14]:
#인퍼런스 설정
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-qa/epoch=0-val_loss=0.45.ckpt


In [15]:
import torch
from transformers import BertConfig, BertForQuestionAnswering

#BERT설정 로드
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

#모델 초기화
model = BertForQuestionAnswering(pretrained_model_config)
#체크포인트 주입
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

#평가모드로 전환
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [16]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [17]:
#인퍼런스
def inference_fn(question, context):
    if question and context:
        
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
       )
        #질문을 토큰화하고 인덱싱하되 max_query_length보다 길면 자르기

        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.max_seq_length,
            return_token_type_ids=True,
        )
        #질문(truncated_query)을 지문(context)과 함께 토큰화하고 인뎅식하되 길이가
        ##max_query_length보다 길면 지문 자르기(truncation="only_second")

        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            #정답의 시작 위치와 관련된 로짓(outputs.start_logits)에서 가장 큰 값이 가리키는 토큰 위치를 알아내기
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            #outputs.end_logits에서 자장 큰 값이 나타내는 위치 찾기
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
            #정답의 시작부터 끝까지 토큰을 이어붙여 정답 만들기
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

In [20]:
inference_fn('한강대교 아래에는 어떤 섬이 있는가?','나미섬은 한강대교 아래의 섬으로 둘레는 모르고 넓이도 모른다.')

{'answer': '나미섬',
 'context': '나미섬은 한강대교 아래의 섬으로 둘레는 모르고 넓이도 모른다.',
 'question': '한강대교 아래에는 어떤 섬이 있는가?'}